In [1]:
from pulp import *

In [3]:
model=LpProblem('Shipping_cost',LpMinimize)

In [6]:
customers=['Ind','can','eur']
products=['tea','coffee','boost']
factory=['fac1','fac2']

In [7]:
keys=[(f,p,c) for f in factory for p in products for c in customers]

In [9]:
var=LpVariable.dicts('shipping_cost',keys,0,None,cat='Integer')


In [10]:
cost_value = [50,80,50,60,90,60,70,90,70,80,50,80,90,60,90,90,70,90]

In [11]:
costs=dict(zip(keys,cost_value))

In [12]:
costs

{('fac1', 'tea', 'Ind'): 50,
 ('fac1', 'tea', 'can'): 80,
 ('fac1', 'tea', 'eur'): 50,
 ('fac1', 'coffee', 'Ind'): 60,
 ('fac1', 'coffee', 'can'): 90,
 ('fac1', 'coffee', 'eur'): 60,
 ('fac1', 'boost', 'Ind'): 70,
 ('fac1', 'boost', 'can'): 90,
 ('fac1', 'boost', 'eur'): 70,
 ('fac2', 'tea', 'Ind'): 80,
 ('fac2', 'tea', 'can'): 50,
 ('fac2', 'tea', 'eur'): 80,
 ('fac2', 'coffee', 'Ind'): 90,
 ('fac2', 'coffee', 'can'): 60,
 ('fac2', 'coffee', 'eur'): 90,
 ('fac2', 'boost', 'Ind'): 90,
 ('fac2', 'boost', 'can'): 70,
 ('fac2', 'boost', 'eur'): 90}

In [13]:
demand_keys=[(p,c) for p in products for c in customers]

In [14]:
demand_value=[90,120,78,65,450,52,700,40,500]

In [15]:
demand=dict(zip(demand_keys,demand_value))

In [16]:
demand

{('tea', 'Ind'): 90,
 ('tea', 'can'): 120,
 ('tea', 'eur'): 78,
 ('coffee', 'Ind'): 65,
 ('coffee', 'can'): 450,
 ('coffee', 'eur'): 52,
 ('boost', 'Ind'): 700,
 ('boost', 'can'): 40,
 ('boost', 'eur'): 500}

In [17]:
model+=lpSum(var[(f,p,c)]*costs[(f,p,c)] for f in factory for p in products for c in customers)

In [18]:
# constriants 

In [19]:
model+=lpSum(var[('fac1',p,c)] for p in products for c in customers) <=1500
model+=lpSum(var[('fac2',p,c)] for p in products for c in customers) <=2500

In [22]:
for p in products:
    for c in customers:
        model+= (var[('fac1',p,c)]+var[('fac2',p,c)]) >= demand[(p,c)]

In [23]:
model.solve()

1

In [24]:
value(model.objective)

135220.0

In [31]:
for i in var:
    print('{} shipping cost is {}'.format(i,var[i].varValue))


('fac1', 'tea', 'Ind') shipping cost is 90.0
('fac1', 'tea', 'can') shipping cost is 0.0
('fac1', 'tea', 'eur') shipping cost is 78.0
('fac1', 'coffee', 'Ind') shipping cost is 65.0
('fac1', 'coffee', 'can') shipping cost is 0.0
('fac1', 'coffee', 'eur') shipping cost is 52.0
('fac1', 'boost', 'Ind') shipping cost is 700.0
('fac1', 'boost', 'can') shipping cost is 0.0
('fac1', 'boost', 'eur') shipping cost is 500.0
('fac2', 'tea', 'Ind') shipping cost is 0.0
('fac2', 'tea', 'can') shipping cost is 120.0
('fac2', 'tea', 'eur') shipping cost is 0.0
('fac2', 'coffee', 'Ind') shipping cost is 0.0
('fac2', 'coffee', 'can') shipping cost is 450.0
('fac2', 'coffee', 'eur') shipping cost is 0.0
('fac2', 'boost', 'Ind') shipping cost is 0.0
('fac2', 'boost', 'can') shipping cost is 40.0
('fac2', 'boost', 'eur') shipping cost is 0.0


In [32]:
# production scheduling

In [33]:
param=pd.read_excel('C:\\documents\\study material\\data science\\supply chain\\Udemy - RA Data Science and Supply Chain analytics. A-Z with Python 2020-9\\14. Linear Programing  in python\\20.1 assignment_ps.xlsx')

<IPython.core.display.Javascript object>

In [34]:
param

,Unnamed: 0,demand,fixed cost,var,storage cost,Capacity
0,period 1,100,2000,40,1,5000
1,period 2,4000,2000,50,1,6000
2,period 3,2500,2000,80,1,4000
3,period 4,4000,2000,60,1,12000
4,period 5,6000,2000,50,1,4000
5,period 6,100,2000,60,1,4000
6,period 7,250,3000,20,1,8000
7,period 8,5000,3000,40,1,4000
8,period 9,6000,3000,90,1,4000
9,period 10,4000,3000,30,1,7000


In [143]:
list1=[i for i in range(1,13)]

In [144]:
param['t']=list1

In [145]:
param=param.set_index('t')

In [146]:
param=param.rename(columns={'Unnamed: 0':'Period'})

In [147]:
model=LpProblem('Prod',LpMinimize)

In [148]:
list2=[i for i in range(0,13)]

In [149]:
inventory=LpVariable.dicts('inv',list2,0,None,'Integer')
inventory[0]=0

In [150]:
binary=LpVariable.dicts('bin',list1,0,None,'Binary')

In [151]:
production=LpVariable.dicts('prod',list1,0,None,'Integer')

In [152]:
time=list1

In [153]:
# objective

In [154]:
time

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [155]:
model+= lpSum([inventory[t]*param.loc[t,'storage cost']+production[t]*param.loc[t,'var']+binary[t]*param.loc[t,'fixed cost'] for t in time ])

In [156]:
for t in time:
    model+= production[t]-inventory[t]+inventory[t-1] >= param.loc[t,'demand']
    model+= production[t] <= binary[t]*param.loc[t,'Capacity']

In [157]:
model.solve()

1

In [158]:
value(model.objective)

1553950.0

In [159]:
for v in model.variables():
    print(v,v.varValue)

bin_1 1.0
bin_10 1.0
bin_11 1.0
bin_12 0.0
bin_2 1.0
bin_3 0.0
bin_4 1.0
bin_5 1.0
bin_6 1.0
bin_7 1.0
bin_8 1.0
bin_9 0.0
inv_1 4900.0
inv_10 4900.0
inv_11 8000.0
inv_12 0.0
inv_2 6900.0
inv_3 4400.0
inv_4 2000.0
inv_5 0.0
inv_6 1150.0
inv_7 8900.0
inv_8 7900.0
inv_9 1900.0
prod_1 5000.0
prod_10 7000.0
prod_11 4000.0
prod_12 0.0
prod_2 6000.0
prod_3 0.0
prod_4 1600.0
prod_5 4000.0
prod_6 1250.0
prod_7 8000.0
prod_8 4000.0
prod_9 0.0


In [160]:
for i in time:
    print(production[i],production[i].varValue)

prod_1 5000.0
prod_2 6000.0
prod_3 0.0
prod_4 1600.0
prod_5 4000.0
prod_6 1250.0
prod_7 8000.0
prod_8 4000.0
prod_9 0.0
prod_10 7000.0
prod_11 4000.0
prod_12 0.0


In [164]:
optimization_data = pd.DataFrame({'demand': param['demand'],
                                 'production':[production[i].varValue for i in production],
                                 'inventory':[inventory[i].varValue for i in time]
                                 })

<IPython.core.display.Javascript object>

In [165]:
optimization_data

,demand,production,inventory
t,,,
1,100,5000.0,4900.0
2,4000,6000.0,6900.0
3,2500,0.0,4400.0
4,4000,1600.0,2000.0
5,6000,4000.0,0.0
6,100,1250.0,1150.0
7,250,8000.0,8900.0
8,5000,4000.0,7900.0
9,6000,0.0,1900.0
